# FVGP Single Task Notebook
In this notebook we will go through a few features of fvGP. We will be primarily concerned with regression over a single dimension output and single task. See the multiple_task_test_notebook.ipynb for single dimension and multiple task example. The extension to multiple dimensions is straight forward.

## Import fvgp and relevant libraries

In [ ]:
import fvgp
from fvgp import gp
import numpy as np
import matplotlib.pyplot as plt

## Defining some input data and testing points

In [ ]:
def function(x):
    return np.sin(1.1 * x)+np.cos(0.5 * x)
x_data = np.linspace(-2*np.pi, 10*np.pi,50).reshape(-1,1)
y_data = function(x_data)
x_pred = np.linspace(-2*np.pi, 10 * np.pi, 1000)

## Setting up the fvgp single task object
NOTE: The input data need to be given in the form (N x input_space_dim). The output can either be a N array or N x 1 array where N is the number of data points. See help(gp.GP) for more information.

In [ ]:
obj = gp.GP(1, x_data,y_data, init_hyperparameters = np.array([10,10]),use_inv = False)

## Training our gaussian process regression on given data


In [ ]:
hyper_param_bounds = np.array([[0.001, 5.],[ 0.001, 100]])
##this will block the main thread, even if you use "hgdl", another option is "global" or "local"
obj.train(hyper_param_bounds, method = "hgdl")

## Looking the posterior mean at the test points

In [ ]:
post_mean= obj.posterior_mean(x_pred.reshape(-1,1))["f(x)"]
post_var= obj.posterior_covariance(x_pred.reshape(-1,1))["v(x)"]

In [ ]:
plt.plot(x_pred, post_mean, label='gp interpolation')
plt.scatter(x_data, y_data, label='data')
plt.plot(x_pred,function(x_pred), label = 'ground truth')
plt.fill_between(x_pred, post_mean + 3.0 *np.sqrt(post_var),post_mean - 3.0 * np.sqrt(post_var), color = 'grey', alpha = 0.5)
plt.legend()

# Training Asynchronously 

In [ ]:
obj = gp.GP(1, x_data,y_data, init_hyperparameters = np.array([10,10]),
                            variances = np.zeros(y_data.reshape(-1,1).shape))


In [ ]:
hyper_param_bounds = np.array([[0.0001, 100], [ 0.0001, 100]])

In [ ]:
async_obj = obj.train_async(hyper_param_bounds)

## Updating asynchronously
Updates hyperparameters to current optimization values

In [ ]:
obj.update_hyperparameters(async_obj)

## Killing training 


In [ ]:
obj.kill_training(async_obj)

## Looking at the posterior mean at the test points

In [ ]:
post_mean= obj.posterior_mean(x_pred.reshape(-1,1))['f(x)']

In [ ]:
plt.plot(x_pred, post_mean, label='interpolation')
plt.scatter(x_data, y_data, label='data')
plt.plot(x_pred, function(x_pred), label='ground truth')
plt.fill_between(x_pred, post_mean + 3.0 *np.sqrt(post_var),post_mean - 3.0 * np.sqrt(post_var), color = 'grey', alpha = 0.5)
plt.legend()

# Custom Kernels

In [ ]:
def kernel_l1(x1,x2, hp, obj):
    ################################################################
    ###standard anisotropic kernel in an input space with l1########
    ################################################################
    d1 = abs(np.subtract.outer(x1[:,0],x2[:,0])) 
    return hp[0] * np.exp(-d1/hp[1])

In [ ]:
obj = gp.GP(1, x_data,y_data,
                init_hyperparameters = np.array([10,10]),
                variances = np.zeros(y_data.shape),
                gp_kernel_function = kernel_l1)

## Training our gaussian process regression on given data


In [ ]:
hyper_param_bounds = np.array([[0.0001, 1000],[ 0.0001, 1000]])
obj.train(hyper_param_bounds)

## Looking the posterior mean at the test points

In [ ]:
post_mean= obj.posterior_mean(x_pred.reshape(-1,1))["f(x)"]

In [ ]:
plt.plot(x_pred, post_mean, label='interpolation')
plt.scatter(x_data, y_data, label='data')
plt.plot(x_pred, function(x_pred), label='ground truth')
plt.fill_between(x_pred, post_mean + 3.0 *np.sqrt(post_var),post_mean - 3.0 * np.sqrt(post_var), color = 'grey', alpha = 0.5)
plt.legend()

# Prior Mean Functions
### NOTE: The prior mean function must return a 1d vector, e.g., (100,)

In [ ]:
def example_mean(gp_obj,x,hyperparameters):
    return np.ones(len(x))

In [ ]:
obj = gp.GP(1, x_data,y_data,init_hyperparameters = np.array([10,10]),
                            variances = np.zeros(y_data.shape),
                            gp_mean_function = example_mean)

## Training our gaussian process regression on given data


In [ ]:
hyper_param_bounds = np.array([[0.0001, 1000],[ 0.0001, 1000]])
obj.train(hyper_param_bounds)

## Looking the posterior mean at the test points

In [ ]:
post_mean= obj.posterior_mean(x_pred.reshape(-1,1))["f(x)"]

In [ ]:
plt.plot(x_pred, post_mean, label='interpolation')
plt.scatter(x_data, y_data, label='data')
plt.plot(x_pred, function(x_pred), label='ground truth')
plt.fill_between(x_pred, post_mean + 3.0 *np.sqrt(post_var),post_mean - 3.0 * np.sqrt(post_var), color = 'grey', alpha = 0.5)
plt.legend()